## Route shape matching
### Packages

- Import standard Python packages.
- Import local modules for route shape matching.

In [1]:
# Import standard packages
import os
import sys
import pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely.geometry import LineString
import numpy as np
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

# Import local module
sys.path.append('../src/')
import route_shape_matching.rm_utilities as rmu

2022-01-11 23:04:15 INFO: WMATA Route Shape Matching.


### Directory & Parameters
- Set directory to read the geospatial files.
- Set or import parameters from local modules such as coordinate reference system. 

In [2]:
# Directory
data_dir = os.path.normpath(r'..\\data')
os.makedirs(os.path.join(data_dir, 'processed', 'route_shapes'), exist_ok = True)
output_dir = os.path.join(data_dir, 'processed', 'route_shapes')

# Parameters
CRS = rmu.CRS
CRS_METER = rmu.CRS_METER

# GIS route line geometries 
gis_lines_geojson = os.path.join(data_dir,
                                 'external',
                                 'gis_bsrt_line',
                                 'gis_bsrt_line.geojson')

gtfs_shapes_text = os.path.join(data_dir,
                                'external',
                                'gtfs_20210903',
                                'shapes.txt')

stop_seq_csv = os.path.join(data_dir, 
                            'external', 
                            'stop_sequence', 
                            'bus_net_stop_sequence.csv')

### Read OSM data
- Read OSM network as graph object.
- Extract edges and nodes with their spatial indexes for faster queries.

In [3]:
G = ox.load_graphml(os.path.join(data_dir,
                                 'processed',
                                 'osm_graph', 
                                 '20220104_osm_graph.graphml'))

# Nodes and edges and their spatial indexes from graph
# Nodes and edges from graph
nodes, edges, node_sindex, edge_sindex = rmu.get_graph_nodes_edges_sindex(G)

2022-01-11 23:05:58 INFO: OSM graph converted to nodes (333,606) and edges (824,767) GeoDataFrames.
2022-01-11 23:06:53 INFO: Spatial indexes returned for 333,606 nodes and 824,767 edges.


### Read data
- Read all GIS route lines.
- Read all GTFS shapes.
- Read all stop sequences.

In [4]:
gis_routes = rmu.read_all_gis_route_lines(gis_lines_geojson, CRS)
all_shapes = rmu.read_gtfs_shapes(gtfs_shapes_text, CRS)
all_stop_seq = rmu.read_all_routes_stop_sequences(stop_seq_csv, CRS)

2022-01-11 23:07:18 INFO: Route line geometries returned for 1,147 unique route patterns.
2022-01-11 23:07:36 INFO: Route shape geometries returned for 558 route patterns with 640,442 points.
2022-01-11 23:07:36 INFO: Stop sequences returned for 1,748 route patterns.


### Selection parameter

In [5]:
route_pattern_id = 'G801'
query_month = 20210903

### Select data for candidate route pattern
- Select GIS route line and GTFS shape points for candidate route pattern.
- Create a line geometry connecting the shape points.

In [6]:
gis_route = rmu.get_single_gis_route_line(gis_routes, route_pattern_id, query_month)
shapes = rmu.get_single_route_gtfs_shapes(all_shapes, route_pattern_id)
shapes_line = rmu.get_line_geom_from_shapes(shapes, crs = CRS)
route_stop_seq = rmu.get_single_route_stop_sequences(all_stop_seq, route_pattern_id)

2022-01-11 23:07:37 INFO: Route line geometries returned for G801.
2022-01-11 23:07:37 INFO: Route shape geometries returned for G801 with 879 points.
2022-01-11 23:07:37 INFO: Route line geometries returned for G801 using 879 points.
2022-01-11 23:07:37 INFO: Stop sequences returned for G801 with 44 stops.


### Create route buffer
- Create a 25 meters buffer around the route line (both GTFS shapes and GIS line).
- Union both buffers to create more accurate clipper for OSM edges.

In [7]:
# Create buffers for GTFS lines and GIS lines
shapes_line_buffer = rmu.create_buffer(shapes_line, 
                                       buffer_in_meter = 25, 
                                       output_crs = CRS, 
                                       sort_columns = ['shape_id'])

gis_route_buffer = rmu.create_buffer(gis_route, 
                                     buffer_in_meter = 25, 
                                     output_crs = CRS, 
                                     sort_columns = ['shape_id'])[['shape_id', 'geometry']]
buffer_geom = rmu.create_union_of_buffer_geom(shapes_line_buffer, gis_route_buffer, only_geom = True)

2022-01-11 23:07:38 INFO: Route buffer geometry created.


### Create hardclipped graph and return edges
- Select nodes inside the buffer.
- Create a subgraph using the select nodes.
- Get edges of the subgraph.
- Clip the edges by route buffer geometry.
- Select candidate edges comparing the clipped length to actual length.
- Add the nearest edges to start and end point of the route.
- Create the largest connected graph with the final candidate edge list.

In [8]:
g_h, ge_h = rmu.get_hardclipped_edges(G, shapes, edges, buffer_geom, return_nodes=False)

2022-01-11 23:08:28 INFO: 814 edges (of 824,767 total edges) found inside the buffer.
2022-01-11 23:08:28 INFO: 439 nodes returned.
2022-01-11 23:08:30 INFO: OSM graph converted to nodes (197) and edges (352) GeoDataFrames.
2022-01-11 23:08:30 INFO: 352 edges returned after final clipping.


### Process shape points
- Using the subgraph, find the nearest edge to the start and end point of each shape-shape pair.
- Correct orientation of returned nearest edges using bearing values.
- Get the shortest path for each shape-shape in sequential order and keep unique segments.
- Get the nodes in correct order using forward search method.
- Construct the route path.
- Merge stop sequences.
- Create new columns.
- Do final processing and export.

In [9]:
route_path = rmu.get_route_path(shapes, g_h, ge_h)
route_path_with_stops = rmu.merge_stop_sequences(route_stop_seq, route_path)
final_route_edges, n, repeated_edges = rmu.process_route_edges_outputs(route_path_with_stops, route_stop_seq, route_pattern_id, crs = CRS)
rmu.export_final_network_edges(output_dir, final_route_edges, route_pattern_id, overwrite = True)

2022-01-11 23:08:59 INFO: Orientation for origin edge corrected for 927 shape-shape pairs.
2022-01-11 23:08:59 INFO: Orientation for destination edge corrected for 1,111 shape-shape pairs.
2022-01-11 23:08:59 INFO: Route path constructed with 159 edges.
2022-01-11 23:08:59 INFO: 44 stops merged with route edges.
2022-01-11 23:08:59 INFO: Final processing completed. 159 matched route edges returned.
2022-01-11 23:08:59 WARNING: Route path contains repeated edges. Expected 157, returned 159.
2022-01-11 23:08:59 WARNING: 2 edges have repetitions. Edges are ['49730804-49730800', '49730800-49730804'].
2022-01-11 23:08:59 INFO: G801_FINAL_BUS_NET_LINK.geojson exported to ..\data\processed\route_shapes\G801.
